## 추론용

### 1. 모듈 불러오기

In [1]:
import os
import copy
import torch
from tqdm import tqdm
import pandas as pd
import detectron2
from detectron2.data import detection_utils as utils
from detectron2.utils.logger import setup_logger
setup_logger()

from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

# 시각화 
from detectron2.utils.visualizer import Visualizer, ColorMode
import cv2

### 2. 데이터 등록

#### 리소스 등록 및 경로 설정

In [2]:
coco_dataset_test = 'coco_trash_test'
path_dataset = '/data/ephemeral/home/dataset/'
path_output_eval = './output_eval'
path_weights = 'model_final.pth'

path_model_pretrained = 'COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml'

In [3]:
def setup_cfg():
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file(path_model_pretrained))
    cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, path_weights)
    cfg.DATASETS.TEST = (coco_dataset_test,)

    cfg.DATALOADER.NUM_WOREKRS = 4
    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 10
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.3
    return cfg

#### 데이터셋 등록

In [4]:
if coco_dataset_test not in DatasetCatalog.list():
    register_coco_instances(coco_dataset_test, {}, path_dataset + 'test.json', path_dataset)

cfg = setup_cfg()

In [5]:
# model
predictor = DefaultPredictor(cfg)

# mapper - input data를 어떤 형식으로 return할지
def MyMapper(dataset_dict):
    
    dataset_dict = copy.deepcopy(dataset_dict)
    image = utils.read_image(dataset_dict['file_name'], format='BGR')
    
    dataset_dict['image'] = image
    
    return dataset_dict

# test loader
test_loader = build_detection_test_loader(cfg, coco_dataset_test, MyMapper)

[10/12 15:45:18 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from ./output/model_final.pth ...
[10/12 15:45:19 d2.data.datasets.coco]: Loaded 4871 images in COCO format from /data/ephemeral/home/dataset/test.json
[10/12 15:45:19 d2.data.build]: Distribution of instances among all 10 categories:
|   category    | #instances   |  category   | #instances   |  category  | #instances   |
|:-------------:|:-------------|:-----------:|:-------------|:----------:|:-------------|
| General trash | 0            |    Paper    | 0            | Paper pack | 0            |
|     Metal     | 0            |    Glass    | 0            |  Plastic   | 0            |
|   Styrofoam   | 0            | Plastic bag | 0            |  Battery   | 0            |
|   Clothing    | 0            |             |              |            |              |
|     total     | 0            |             |              |            |              |
[10/12 15:45:19 d2.data.common]: Serializing the d

In [11]:
# output 뽑은 후 sumbmission 양식에 맞게 후처리 
prediction_strings = []
file_names = []

class_num = 10

for data in tqdm(test_loader):
    
    prediction_string = ''
    
    data = data[0]
    
    outputs = predictor(data['image'])['instances']
    
    targets = outputs.pred_classes.cpu().tolist()
    boxes = [i.cpu().detach().numpy() for i in outputs.pred_boxes]
    scores = outputs.scores.cpu().tolist()
    
    for target, box, score in zip(targets,boxes,scores):
        prediction_string += (str(target) + ' ' + str(score) + ' ' + str(box[0]) + ' ' 
        + str(box[1]) + ' ' + str(box[2]) + ' ' + str(box[3]) + ' ')
    
    prediction_strings.append(prediction_string)
    file_names.append(data['file_name'].replace(path_dataset,''))

    # 시각화 코드
    v = Visualizer(data['image'], MetadataCatalog.get(cfg.DATASETS.TEST[0]), scale=1.2)
    out = v.draw_instance_predictions(outputs.to("cpu"))

    result_file = os.path.join(cfg.OUTPUT_DIR, f"visualization_{data['file_name'].split('/')[-1]}")
    cv2.imwrite(result_file, out.get_image()[:, :, ::-1])

submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.OUTPUT_DIR, f'submission_det_{path_model_pretrained.split("/")[1]}.csv'), index=None)

100%|██████████| 4871/4871 [20:22<00:00,  3.98it/s]
